In [12]:
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
embedding = OllamaEmbeddings(model="llama3")

In [13]:
persist_directory = './chroma/'

In [14]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [15]:
print(vectordb._collection.count())

53


In [16]:
from langchain_community.llms import Ollama
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [17]:
metadata_field_info = [
    AttributeInfo(
        name="Header",
        description="This is the header of the section from which this text originates",
        type="string",
    ),
    AttributeInfo(
        name="source",
        description="Path of the PDF file from which this chunk is derived. It is possible to observe the hash that symbolizes the name of the PDF, in addition to the number of questions it contains. The PDFs are medical scientific articles in the field of medicine. The path format is 'articles/hash(n).pdf', where 'hash' represents the PDF name, 'n' represents the number of related questions, and '.pdf' represents the file extension.",
        type="string",
    ),
]

document_content_description = "Articles"

In [18]:
llm = Ollama(
    model="llama3",
    temperature = 0
)

retriever = vectordb.as_retriever()

In [19]:
question = "What is the main subject of the article?"
docs_ss = vectordb.similarity_search(question,k=3)
len(docs_ss)

3

In [20]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import pipeline

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

section_counts = {
        "Introduction": 0,
        "Background": 0,
        "Methods": 0,
        "Results": 0,
        "Discussion": 0,
        "Conclusion": 0
    }

In [21]:
# questions_data = [
#     {"id": "112338", "questiontext": "<p>Cette étude vise à démontrer la supériorité du pembrolizumab (une ou plusieurs réponses possibles) :</p>"},
#     {"id": "112339", "questiontext": "<p>L'allocation aléatoire (randomisation) du traitement à l'étude vise à (une seule réponse possible) :</p>"},
#     {"id": "112340", "questiontext": "<p>L’absence de double insu (une ou plusieurs réponses possibles) :</p>"},
#     {"id": "112341", "questiontext": "<p>Parmi les critères de jugement étudiés dans cet essai, lequel est le plus cliniquement pertinent ?</p>"},
#     {"id": "112342", "questiontext": "<p>Dans cette étude, les sources de multiplicité de comparaisons statistiques sont (une ou plusieurs réponses possibles) :</p>"},
#     {"id": "112343", "questiontext": "<p>Le risque alpha attribué à la seconde analyse intermédiaire pour la survie globale est de 0,005 en unilatéral. Il correspond à un risque alpha bilatéral de (une seule réponse exacte) :</p>"},
#     {"id": "112344", "questiontext": "<p>Parmi les propositions suivantes concernant la première analyse intermédiaire, laquelle est exacte ?</p>"},
#     {"id": "112345", "questiontext": "<p>La deuxième analyse intermédiaire devait avoir lieu (une ou plusieurs réponses possibles) :</p>"},
#     {"id": "112346", "questiontext": "<p>Dans la figure 1, il existe un nombre important de censures (identifiées par la zone grisée au départ de la courbe) dans le groupe contrôle « ipilimumab » au J0. Parmi les propositions suivantes concernant ces censures, laquelle (lesquelles) est (sont) exacte(s) ?</p>"},
#     {"id": "112347", "questiontext": "<p>Le résultat obtenu sur la survie globale avec l’administration du pembrolizumab toutes les 2 semaines (une ou plusieurs réponses possibles) :</p>"},
#     {"id": "112348", "questiontext": "<p>A l’issue de la 2<sup>ème</sup> analyse intermédiaire (une ou plusieurs réponses possibles) :</p>"},
#     {"id": "112349", "questiontext": "<p>Parmi les propositions suivantes concernant la figure 1 panneau B, laquelle (lesquelles) est (sont) exacte(s) ?</p>"},
#     {"id": "112350", "questiontext": "<p>Parmi les propositions suivantes concernant les analyses en sous-groupes sur la survie globale (figure 2 panneau b), laquelle (lesquelles) est (sont) exacte(s) ?</p>"},
#     {"id": "112351", "questiontext": "<p>La limite la plus importante de cette étude est (une seule réponse) :</p>"},
#     {"id": "112352", "questiontext": "<p>Les résultats de cet essai incitent à proposer chez les patients présentant un mélanome au stade avancé (une seule réponse possible) :</p>"}
# ]

# # Dados das respostas
# answers_data = [
#     {"id": "502997", "question": "112338", "answer": "<p>toutes les 2 semaines sur la survie globale comparativement à l’ipilimumab</p>", "fraction": "0.5000000"},
#     {"id": "502998", "question": "112338", "answer": "<p>toutes les 3 semaines sur la survie globale comparativement à l’ipilimumab</p>", "fraction": "0.5000000"},
#     {"id": "502999", "question": "112338", "answer": "<p>toutes les 2 semaines sur la survie sans progression comparativement à l’ipilimumab</p>", "fraction": "0.5000000"},
#     {"id": "503000", "question": "112338", "answer": "<p>toutes les 3 semaines sur la survie sans progression comparativement à l’ipilimumab</p>", "fraction": "0.5000000"},
#     {"id": "503001", "question": "112338", "answer": "<p>toutes les 2 semaines sur la survie sans progression comparativement au pembrolizumab toutes les 3 semaines</p>", "fraction": "0.0000000"},
#     {"id": "503002", "question": "112339", "answer": "<p>limiter les fluctuations d'échantillonnage</p>", "fraction": "0.0000000"},
#     {"id": "503003", "question": "112339", "answer": "<p>augmenter la puissance statistique de l'essai</p>", "fraction": "0.0000000"},
#     {"id": "503004", "question": "112339", "answer": "<p>rendre l’attribution du traitement indépendante des caractéristiques des patients</p>", "fraction": "1.0000000"},
#     {"id": "503005", "question": "112339", "answer": "<p>contrôler le risque d'erreur statistique de 1ère espèce (alpha)</p>", "fraction": "0.0000000"},
#     {"id": "503006", "question": "112339", "answer": "<p>maintenir la comparabilité des trois groupes tout au long de l'essai</p>", "fraction": "0.0000000"},
#     {"id": "503007", "question": "112340", "answer": "<p>pouvait influencer la mesure du critère de survie globale</p>", "fraction": "0.0000000"},
#     {"id": "503008", "question": "112340", "answer": "<p>pouvait influencer la mesure du critère de survie sans progression</p>", "fraction": "0.5000000"},
#     {"id": "503009", "question": "112340", "answer": "<p>pouvait influencer la prise en charge des patients </p>", "fraction": "0.5000000"},
#     {"id": "503010", "question": "112340", "answer": "<p>pouvait diminuer la puissance de l’étude</p>", "fraction": "0.0000000"},
#     {"id": "503011", "question": "112340", "answer": "<p>remet en cause la validité interne de cet essai</p>", "fraction": "0.5000000"},
#     {"id": "503012", "question": "112341", "answer": "<p>taux de réponse au traitement</p>", "fraction": "0.0000000"},
#     {"id": "503013", "question": "112341", "answer": "<p>survie globale</p>", "fraction": "1.0000000"},
#     {"id": "503014", "question": "112341", "answer": "<p>survie sans progression</p>", "fraction": "0.0000000"},
#     {"id": "503015", "question": "112341", "answer": "<p>durée de la réponse</p>", "fraction": "0.0000000"},
#     {"id": "503016", "question": "112341", "answer": "<p>taux de survie sans progression à 6 mois</p>", "fraction": "0.0000000"},
#     {"id": "503017", "question": "112342", "answer": "<p>duplicité des critères de jugement principaux : survie sans progression et survie globale </p>", "fraction": "0.5000000"},
#     {"id": "503018", "question": "112342", "answer": "<p>réalisation d’analyses intermédiaires</p>", "fraction": "0.5000000"},
#     {"id": "503019", "question": "112342", "answer": "<p>utilisation d’un critère composite</p>", "fraction": "0.0000000"},
#     {"id": "503020", "question": "112342", "answer": "<p>évaluation de 2 posologies du pembrolizumab</p>", "fraction": "0.5000000"},
#     {"id": "503021", "question": "112342", "answer": "<p>participation à l’étude de plusieurs centres</p>", "fraction": "0.0000000"},
#     {"id": "503022", "question": "112343", "answer": "<p>0,01</p>", "fraction": "1.0000000"},
#     {"id": "503023", "question": "112343", "answer": "<p>0,0025</p>", "fraction": "0.0000000"},
#     {"id": "503024", "question": "112343", "answer": "<p>0,05</p>", "fraction": "0.0000000"},
#     {"id": "503025", "question": "112343", "answer": "<p>0,1</p>", "fraction": "0.0000000"},
#     {"id": "503026", "question": "112343", "answer": "<p>0,005</p>", "fraction": "0.0000000"},
#     {"id": "503027", "question": "112344", "answer": "<p>la supériorité du pembrolizumab sur l’ipilimumab est observée pour la survie globale</p>", "fraction": "1.0000000"},
#     {"id": "503028", "question": "112344", "answer": "<p>la non-infériorité du pembrolizumab sur l’ipilimumab est observée pour la survie globale</p>", "fraction": "0.0000000"},
#     {"id": "503029", "question": "112344", "answer": "<p>la supériorité du pembrolizumab sur l’ipilimumab est observée pour la survie sans progression</p>", "fraction": "0.0000000"},
#     {"id": "503030", "question": "112344", "answer": "<p>la non-infériorité du pembrolizumab sur l’ipilimumab est observée pour la survie sans progression</p>", "fraction": "0.0000000"},
#     {"id": "503031", "question": "112344", "answer": "<p>aucune de ces réponses</p>", "fraction": "0.0000000"},
#     {"id": "503032", "question": "112345", "answer": "<p>lorsque 80 % des événements de survie globale avaient été observés </p>", "fraction": "0.5000000"},
#     {"id": "503033", "question": "112345", "answer": "<p>lorsque 80 % des événements de survie sans progression avaient été observés</p>", "fraction": "0.0000000"},
#     {"id": "503034", "question": "112345", "answer": "<p>lorsque 90 % des événements de survie globale avaient été observés</p>", "fraction": "0.0000000"},
#     {"id": "503035", "question": "112345", "answer": "<p>lorsque 90 % des événements de survie sans progression avaient été observés</p>", "fraction": "0.5000000"},
#     {"id": "503036", "question": "112345", "answer": "<p>lorsque 100 % des événements de survie globale avaient été observés</p>", "fraction": "0.0000000"},
#     {"id": "503037", "question": "112346", "answer": "<p>elles correspondent aux sorties d’étude avant la première administration du traitement </p>", "fraction": "1.0000000"},
#     {"id": "503038", "question": "112346", "answer": "<p>elles correspondent à des sorties d’étude à l’initiative des patients</p>", "fraction": "0.0000000"},
#     {"id": "503039", "question": "112346", "answer": "<p>elles correspondent à des sorties d’étude en rapport avec un effet indésirable du traitement </p>", "fraction": "0.0000000"},
#     {"id": "503040", "question": "112346", "answer": "<p>elles correspondent à des événements de progression de la maladie</p>", "fraction": "0.0000000"},
#     {"id": "503041", "question": "112346", "answer": "<p>elles ne remettent pas en cause la validité des résultats observés dans cet essai</p>", "fraction": "0.5000000"},
#     {"id": "503042", "question": "112347", "answer": "<p>était en faveur du pembrolizumab comparativement à l’ipilimumab </p>", "fraction": "1.0000000"},
#     {"id": "503043", "question": "112347", "answer": "<p>ne permettait pas de conclure à une différence avec l’ipilimumab</p>", "fraction": "0.0000000"},
#     {"id": "503044", "question": "112347", "answer": "<p>était en faveur de l’ipilimumab comparativement au pembrolizumab</p>", "fraction": "0.0000000"},
#     {"id": "503045", "question": "112347", "answer": "<p>était en faveur du pembrolizumab toutes les 3 semaines comparativement à l’ipilimumab</p>", "fraction": "0.0000000"},
#     {"id": "503046", "question": "112347", "answer": "<p>était en faveur du pembrolizumab toutes les 3 semaines comparativement au pembrolizumab toutes les 2 semaines</p>", "fraction": "0.0000000"},
#     {"id": "503047", "question": "112348", "answer": "<p>l’intervalle de confiance de la survie globale du pembrolizumab toutes les 3 semaines était compris entre 9,4 et 17,3 mois </p>", "fraction": "1.0000000"},
#     {"id": "503048", "question": "112348", "answer": "<p>le nombre de décès observés sous pembrolizumab toutes les 3 semaines était de 70</p>", "fraction": "1.0000000"},
#     {"id": "503049", "question": "112348", "answer": "<p>le nombre de décès observés sous pembrolizumab toutes les 2 semaines était de 70</p>", "fraction": "0.0000000"},
#     {"id": "503050", "question": "112348", "answer": "<p>le nombre de décès observés sous ipilimumab était de 74</p>", "fraction": "1.0000000"},
#     {"id": "503051", "question": "112348", "answer": "<p>le rapport de risque (HR) était en faveur de l’ipilimumab comparativement au pembrolizumab toutes les 3 semaines</p>", "fraction": "0.0000000"},
#     {"id": "503052", "question": "112349", "answer": "<p>la différence de survie globale entre pembrolizumab toutes les 3 semaines et ipilimumab était statistiquement significative</p>", "fraction": "1.0000000"},
#     {"id": "503053", "question": "112349", "answer": "<p>la différence de survie globale entre pembrolizumab toutes les 3 semaines et ipilimumab n’était pas statistiquement significative</p>", "fraction": "0.0000000"},
#     {"id": "503054", "question": "112349", "answer": "<p>la différence de survie globale entre pembrolizumab toutes les 3 semaines et ipilimumab ne peut pas être estimée</p>", "fraction": "0.0000000"},
#     {"id": "503055", "question": "112349", "answer": "<p>la différence de survie sans progression entre pembrolizumab toutes les 3 semaines et ipilimumab était statistiquement significative</p>", "fraction": "0.0000000"},
#     {"id": "503056", "question": "112349", "answer": "<p>la différence de survie sans progression entre pembrolizumab toutes les 3 semaines et ipilimumab n’était pas statistiquement significative</p>", "fraction": "0.0000000"},
#     {"id": "503057", "question": "112350", "answer": "<p>le sexe féminin est associé à un avantage significatif en faveur du pembrolizumab </p>", "fraction": "1.0000000"},
#     {"id": "503058", "question": "112350", "answer": "<p>les patients de plus de 65 ans sont associés à un avantage significatif en faveur du pembrolizumab</p>", "fraction": "0.0000000"},
#     {"id": "503059", "question": "112350", "answer": "<p>le sexe masculin est associé à un avantage significatif en faveur de l’ipilimumab</p>", "fraction": "0.0000000"},
#     {"id": "503060", "question": "112350", "answer": "<p>les patients ayant un mauvais état général (PS &gt; 1) sont associés à un avantage significatif en faveur de l’ipilimumab</p>", "fraction": "0.0000000"},
#     {"id": "503061", "question": "112350", "answer": "<p>les patients ayant un bon état général (PS = 0) sont associés à un avantage significatif en faveur du pembrolizumab</p>", "fraction": "1.0000000"},
#     {"id": "503062", "question": "112351", "answer": "<p>l’absence de double insu</p>", "fraction": "1.0000000"},
#     {"id": "503063", "question": "112351", "answer": "<p>l’absence d’évaluation de la qualité de vie</p>", "fraction": "0.0000000"},
#     {"id": "503064", "question": "112351", "answer": "<p>l’absence de cécité des évaluateurs</p>", "fraction": "0.0000000"},
#     {"id": "503065", "question": "112351", "answer": "<p>l’absence de stratification des patients selon l’IMC</p>", "fraction": "0.0000000"},
#     {"id": "503066", "question": "112351", "answer": "<p>l’absence de comité de surveillance indépendant</p>", "fraction": "0.0000000"},
#     {"id": "503067", "question": "112352", "answer": "<p>une chimiothérapie systémique d’emblée dans tous les mélanomes métastatiques</p>", "fraction": "0.0000000"},
#     {"id": "503068", "question": "112352", "answer": "<p>une stratégie de combinaison anti-PD-1 et anti-CTLA-4 chez les patients naïfs de traitement</p>", "fraction": "1.0000000"},
#     {"id": "503069", "question": "112352", "answer": "<p>l’ajout d’un traitement adjuvant par l’ipilimumab après résection d’un mélanome de stade III</p>", "fraction": "0.0000000"},
#     {"id": "503070", "question": "112352", "answer": "<p>une chirurgie systématique des métastases pulmonaires uniques</p>", "fraction": "0.0000000"},
#     {"id": "503071", "question": "112352", "answer": "<p>l’ajout d’une thérapie ciblée BRAF/MEK dans tous les mélanomes métastatiques</p>", "fraction": "0.0000000"}
# ]

In [22]:
# def transform_data(questions_data, answers_data):
#     questions = []
    
#     # Criar um dicionário para mapear id de perguntas para respostas
#     answer_map = {}
#     for answer in answers_data:
#         question_id = answer["question"]
#         if question_id not in answer_map:
#             answer_map[question_id] = []
#         if float(answer["fraction"]) > 0:
#             answer_map[question_id].append(answer["answer"])
    
#     # Criar lista de perguntas formatadas
#     for question in questions_data:
#         q_id = question["id"]
#         article_id = "2c5bee41e4ae9aa65206b113c7b84c8152cd4aed.pdf"  # Assuming article ID remains constant
        
#         quizz_question = question["questiontext"]
        
#         # Obter possíveis respostas para a pergunta
#         possible_answers = answer_map.get(q_id, [])
#         possible_answers_str = ";".join(possible_answers)
        
#         # Obter a chave de respostas corretas (answer_key)
#         answer_key = question.get("answer_key", "")  # Assuming answer_key is provided in the questions_data
        
#         # Montar o dicionário para a pergunta atual
#         question_dict = {
#             "article": article_id,
#             "quizz_question": quizz_question,
#             "possible_answers": possible_answers_str,
#             "answer_key": answer_key
#         }
        
#         # Adicionar à lista de perguntas formatadas
#         questions.append(question_dict)
    
#     return questions

# # Uso da função para transformar os dados
# questions = transform_data(questions_data, answers_data)

# print(questions)

In [23]:
# questions = [
#     {
#         "quizz_question": "Pergunta 1: L’objectif principal de l’étude était",
#         "possible_answers": [
#             "d’évaluer la sécurité et l’efficacité du Pantoprazol en préventif en soins intensifs",
#             "d’évaluer l’efficacité seulement du Pantoprazol en préventif en soins intensifs",
#             "d’évaluer la sécurité seulement du Pantoprazol en préventif en soins intensifs",
#             "d’évaluer la sécurité et l’efficacité du Pantoprazol en curatif en soins intensifs",
#             "d’évaluer la sécurité et l’efficacité du Pantoprazol en curatif en soins palliatifs"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 2: Les sujets inclus dans l’étude",
#         "possible_answers": [
#             "présentent un ulcère gastrique chronique",
#             "sont à faible risque de saignement digestif",
#             "présentent au moins un facteur de risque pour un saignement digestif",
#             "sont représentatif des patients rencontrés en médecine générale ambulatoire",
#             "sont extrêmement représentatif des patients tout venant rencontrés en service d’urgence"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 3: Cette étude est une étude",
#         "possible_answers": [
#             "randomisée avec un ratio 3 : 1 (le traitement testé est alloué à 3 sujets pour 1 sujet traité par placebo)",
#             "avec une randomisation locale, par la date de visite",
#             "en simple aveugle",
#             "analysée principalement en per protocole",
#             "contrôlée contre placebo"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 4: Cette étude est protégée contre",
#         "possible_answers": [
#             "le biais de sélection",
#             "les biais de confusion",
#             "le biais de mesure",
#             "le biais de suivi",
#             "le biais d’attrition"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 5: Concernant les critères de jugements de cette étude",
#         "possible_answers": [
#             "le critère de jugement principal était un critère composite",
#             "le critère de jugement principal était un critère intermédiaire",
#             "le critère de jugement principal était la mortalité à 90 jours",
#             "les événements étaient comptabilisés directement par l’investigateur local",
#             "un comité indépendant et en insu a adjudiqué les événements"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 6: Concernant la puissance statistique de cette étude pour le critère de jugement principal",
#         "possible_answers": [
#             "les auteurs avaient prévu une puissance de 90% de détecter une diminution relative de 20% du risque d’événement du critère de jugement principal",
#             "le risque de ne pas voir une différence qui pourtant existe était de 10%",
#             "le risque de type II est de 10%",
#             "si la différence de risque attendue avec le traitement expérimental avait été de 20%, il aurait fallu inclure plus de sujets et/ou observer plus d’événements",
#             "le risque de base attendue était de 25%"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 7: Concernant le plan d’analyse de cette étude",
#         "possible_answers": [
#             "le diagramme de flux n’est pas fourni dans cette publication",
#             "le diagramme de flux permet notamment de vérifier l’absence de déséquilibre entre groupe quant aux perdus de vue",
#             "la majorité des patients évalués pour leur éligibilité pour l’étude correspondaient aux critères d’inclusion de l’étude",
#             "le taux de données manquantes pour l’analyse du critère de jugement principal était anormalement et excessivement élevé",
#             "la durée de suivi a été très inégale entre les deux groupes"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 8: Concernant le déroulement de l’étude",
#         "possible_answers": [
#             "le diagramme de flux correspond à la figure 1, comme cela est l’usage"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 9: Concernant les résultats de cette étude, sur le tableau 1 :",
#         "possible_answers": [
#             "De façon générale, le tableau décrivant les caractéristiques des patients à l’inclusion permettrait de démontrer l’absence de facteurs de confusion dans une étude qui ne serait pas randomisée",
#             "Dans ce tableau 1, les auteurs auraient dû montrer les p values de la comparaison des groupes pour chacune des variables pour démontrer l’absence de facteur de confusion dans l’étude",
#             "Ce tableau 1 permet de s’assurer l’absence de déséquilibre entre les groupes grâce à la randomisation sur des critères connu pour être important dans ce contexte",
#             "Ce tableau 1 permet d’évaluer l’extrapolabilité des résultats de l’étude",
#             "Dans cette étude, à partir de ce tableau 1, les groupes semblent bien équilibrés"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 10: Concernant les résultats de cette étude, sur les critères de jugements principaux et secondaires :",
#         "possible_answers": [
#             "Le traitement testé a démontré une supériorité par rapport au placebo pour le critère de jugement principal",
#             "Le risque de décès à 90 jours dans le groupe traité était de 31.1%",
#             "Le risque de décès à 90 jours dans le groupe contrôle était de 30.4%",
#             "L’étude suggère une possible diminution de 42% du risque de saignement digestif cliniquement important",
#             "Le problème des analyses en risque relatif est que le risque relatif dépend du risque de base"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 11: Concernant le critère de jugement secondaire « au moins 1 événement cliniquement important »",
#         "possible_answers": [
#             "il permet d’évaluer l’efficacité seulement du traitement testé",
#             "il permet d’évaluer la sécurité seulement du traitement testé",
#             "il intègre d’une part des événements dont on espère que le traitement testé diminuera le risque, d’autre part des événements dont on craint que le traitement testé augmente le risque",
#             "il permettrait en théorie de montrer un bénéfice net du traitement",
#             "s'il était supérieur à 1, il serait en faveur d'un bénéfice net."
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 12: Concernant les résultats de cette étude, sur les analyses en sous-groupes sur le critère de jugement principal",
#         "possible_answers": [
#             "les sous-groupes ont été définis a priori",
#             "le fait d’être en choc à la randomisation modifie de façon majeure l’effet du traitement",
#             "le fait d’avoir un score SAPS II >53 ou non semble être associée à une modification de l’effet du traitement",
#             "ces analyses en sous-groupes sont exposées au risque de faux positifs (risque alpha)",
#             "l’estimation de l’effet du traitement chez les patients avec antécédent hépatique était très précise"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 13: Concernant la discussion de cet article",
#         "possible_answers": [
#             "les auteurs comparent leurs résultats à ceux d’autres études",
#             "les auteurs expliquent les limites de leur étude",
#             "les auteurs présentent une (des) hypothèse(s) pour expliquer leurs résultats",
#             "l’origine du saignement digestif n’était pas systématiquement objectivée",
#             "les auteurs suggèrent qu’il serait intéressant d’étudier l’impact de la sévérité du patient sur l’effet du traitement"
#         ]
#     },
#     {
#         "quizz_question": "Pergunta 14: Pour conclure, au sujet de cet article",
#         "possible_answers": [
#             "la conclusion est cohérente au vu des résultats",
#             "le résumé est cohérent au vu de l’article",
#             "les résultats de cette seule étude imposent de prescrire ce médicament à tout patient correspondant aux critères d’inclusion de l’étude",
#             "l’étude était pertinente cliniquement",
#             "la structure d’article IMRAD était respectée"
#         ]
#     }
# ]

In [24]:
questions = [
    {
        "c": "Questão 1",
        "quizz_question": "D'après cet article, quelles sont les réponses exactes ?",
        "possible_answers": [
            "A: L'ergocalciférol peut être synthétisé à partir du 7-deshydrocholestérol au niveau de la peau exposée aux rayonnements ultra-violets B.",
            "B: Après synthèse au niveau de la peau ou à partir des apports alimentaires, la vitamine D est convertie en 25 hydroxyvitamine D dans le foie.",
            "C: La 25 hydroxyvitamine D subit une hydroxylation au niveau du rein qui est régulée par la Parathormone.",
            "D: La 1,25-dihydroxyvitamine D (1,25 (OH2)D peut aussi en faible proportion être synthétisée dans d'autres tissus notamment au niveau du tissu mammaire.",
            "E: Des études observationnelles ont montré que la 25 hydroxyvitamine D pouvait affecter le développement de tumeur."
        ],
        "resposta_correta": "B"
    },
    {
        "c": "Questão 2",
        "quizz_question": "Quel est le schéma de cette étude ?",
        "possible_answers": [
            "A: Etude cas-témoin",
            "B: Etude exposé-non exposé",
            "C: Etude transversale",
            "D: Un essai clinique",
            "E: Une série de cas"
        ],
        "resposta_correta": "A"
    },
    {
        "c": "Questão 3",
        "quizz_question": "Parmi les propositions suivantes, lesquelles font partie des critères d'inclusion de cette étude ?",
        "possible_answers": [
            "A: Etre une femme travaillant pour la fonction publique hospitalière en France",
            "B: Etre née entre 1950 et 1970",
            "C: Avoir complété les auto-questionnaires",
            "D: Avoir bénéficié des prises de sang entre 1990 et 1995",
            "E: Avoir donné des renseignements aux enquêteurs sur le statut ménopausique"
        ],
        "resposta_correta": "D"
    },
    {
        "c": "Questão 4",
        "quizz_question": "Dans cette étude, quel est l'évènement pathologique que présentent les sujets 'cas' et que ne présentent pas les sujets 'temoins'?",
        "possible_answers": [
            "A: le cancer du sein",
            "B: la ménopause",
            "C: le faible taux de vitamine D",
            "D: le surpoids",
            "E: le tabagisme"
        ],
        "resposta_correta": "A"
    },
    {
        "c": "Questão 5",
        "quizz_question": "Dans cette étude, quelles sont les mesures méthodologiques permettant la réduction de biais de sélection ?",
        "possible_answers": [
            "A: La réalisation des analyses des prélèvements sanguins dans un laboratoire n’ayant pas connaissance de la répartition des sujets entre les groupes cas et témoins.",
            "B: La réalisation des analyses des prélèvements sanguins dans un laboratoire n'ayant pas connaissance de la répartition des sujets entre les groupes cas et témoins.",
            "C: L'utilisation de techniques biologiques validées pour mesurer le taux de vitamine D.",
            "D: Le diagnostic de cancer du sein a été confirmé par un médecin",
            "E: Les sujets n'ayant pas rempli les questionnaires ou n'ayant pas eu de prélèvement sanguin ont été exclus de l'analyse"
        ],
        "resposta_correta": "A"
    },
    {
        "c": "Questão 6",
        "quizz_question": "Dans cette étude, quelles sont les mesures méthodologiques prises pour réduire le biais de classement concernant l'exposition ?",
        "possible_answers": [
            "A: La réalisation d'un appariement des deux groupes témoins au groupe des cas sur le statut tabagique.",
            "B: La réalisation d'une analyse stratifiée sur le statut ménopausique pour étudier la relation entre la concentration en 25(OH)D3 et le cancer du sein.",
            "C: La réalisation d'une analyse univariée pour étudier la relation entre la concentration en 25(OH)D3 et le cancer du sein.",
            "D: La réalisation d'un appariement des deux groupes témoins au groupe des cas sur l'utilisation antérieure d'une contraception orale.",
            "E: La réalisation d’une analyse multivariée avec ajustement sur la consommation d'alcool."
        ],
        "resposta_correta": "B"
    },
    {
        "c": "Questão 7",
        "quizz_question": "Dans cette étude, quelles sont les mesures méthodologiques prises pour réduire l'influence des potentiels facteurs de confusion de la relation entre l'apparition du cancer du sein et la vitamine D ?",
        "possible_answers": [
            "A: La concentration de la 25 hyroxyvitamine D3 dans le sang est positivement significativement corrélée avec la concentration de calcium dans le sang chez les femmes en pré-ménopause et en post-ménopause",
            "B: La concentration de la 25 hyroxyvitamine D3 dans le sang est positivement significativement corrélée avec la concentration de parathormone dans le sang",
            "C: La concentration de la 25 hyroxyvitamine D3 dans le sang est positivement significativement corrélée à la concentration d’estradiol dans le sang",
            "D: La concentration de la 25 hyroxyvitamine D3 dans le sang est positivement significativement corrélée à la concentration de progestérone dans le sang",
            "E: Une association significative est mise en évidence entre la survenue de cancer du sein et le surpoids."
        ],
        "resposta_correta": "B"
    },
    {
        "c": "Questão 8",
        "quizz_question": "D’après cet article, quelle est la réponse juste ?",
        "possible_answers": [
            "A: La concentration de la 25 hyroxyvitamine D3 dans le sang est positivement significativement corrélée avec la concentration de calcium dans le sang chez les femmes en pré-ménopause et en post-ménopause",
            "B: La concentration de la 25 hyroxyvitamine D3 dans le sang est positivement significativement corrélée avec la concentration de parathormone dans le sang",
            "C: La concentration de la 25 hyroxyvitamine D3 dans le sang est positivement significativement corrélée à la concentration d’estradiol dans le sang",
            "D: La concentration de la 25 hyroxyvitamine D3 dans le sang est positivement significativement corrélée à la concentration de progestérone dans le sang",
            "E: Une association significative est mise en évidence entre la survenue de cancer du sein et le surpoids."
        ],
        "resposta_correta": "E"
    },
    {
        "c": "Questão 9",
        "quizz_question": "Que déduisez-vous de la relation entre la concentration de la 25 hydroxyvitamine D3 et la présence du cancer du sein à partir des résultats du modèle multivarié §  présenté dans le tableau 2 ?",
        "possible_answers": [
            "A: Une concentration de 25 hydroxyvitamine D3 supérieure à  27.0 ng/ml apparait comme un facteur de risque du cancer du sein.",
            "B: L’association entre la concentration de la  25 hydroxyvitamine D3 et le cancer du sein est statistiquement significative car la valeur du p-value est supérieure à 0.05.",
            "C: Le risque de développer un cancer du sein est significativement plus fort chez les femmes ayant une concentration en 25 hydroxyvitamine D3  inférieure à 19.8 ng/ml que chez les femmes ayant une concentration en 25 hydroxyvitamine D3  supérieure à 27.0 ng/ml.",
            "D: L’association entre la concentration de la  25 hydroxyvitamine D3 et le cancer du sein est statistiquement significative car l’intervalle de confiance exclut la valeur « 1 ».",
            "E: L’association entre la concentration de la  25 hydroxyvitamine D3 et le cancer du sein est statistiquement significative car la borne inférieure de l’intervalle de confiance de l’OR est inférieure à 1."
        ],
        "resposta_correta": "D"
    },
    {
        "c": "Questão 10",
        "quizz_question": "Que déduisez-vous du tableau 3 (une seule réponse juste) ?",
        "possible_answers": [
            "A: Une valeur de p-value <0.05 signifie qu’il n’a pas été montré de différence significative de survenue de cancer du sein en fonction de la concentration de la 25 hydroxyvitamine D3",
            "B: L’association entre la concentration de la  25 hydroxyvitamine D3  et le cancer du sein n’est pas significative après ajustement sur toutes les autres covariables et stratification sur le statut vis-à-vis de la ménopause au moment des prélèvements sanguins et du diagnostic.",
            "C: Un intervalle de confiance de l’Odds Ratio ajusté contenant la valeur 1 indique qu’il n’a pas été détecté de différence significative de survenue du cancer du sein en fonction de la concentration de la  25 hydroxyvitamine D3, avec prise en compte des potentiels facteurs de confusion.",
            "D: La concentration de la  25 hydroxyvitamine D3 est un facteur protecteur pour la survenue de cancer du sein chez les femmes ayant des antécédents familiaux de cancer du sein.",
            "E: L’association entre la concentration de la  25 hydroxyvitamine D3 et le cancer du sein est statistiquement significative car l’intervalle de confiance exclut la valeur « 1 »."
        ],
        "resposta_correta": "C"
    },
    {
        "c": "Questão 11",
        "quizz_question": "D'après cet article, quelles sont les réponses exactes ?",
        "possible_answers": [
            "A: L'ergocalciférol peut être synthétisé à partir du 7-deshydrocholestérol au niveau de la peau exposée aux rayonnements ultra-violets B.",
            "B: Après synthèse au niveau de la peau ou à partir des apports alimentaires, la vitamine D est convertie en 25 hydroxyvitamine D dans le foie.",
            "C: La 25 hydroxyvitamine D subit une hydroxylation au niveau du rein qui est régulée par la Parathormone.",
            "D: La 1,25-dihydroxyvitamine D (1,25 (OH2)D peut aussi en faible proportion être synthétisée dans d'autres tissus notamment au niveau du tissu mammaire.",
            "E: Des études observationnelles ont montré que la 25 hydroxyvitamine D pouvait affecter le développement de tumeur."
        ],
        "resposta_correta": "B"
    },
    {
        "c": "Questão 12",
        "quizz_question": "Quel est le schéma de cette étude ?",
        "possible_answers": [
            "A: Etude cas-témoin",
            "B: Etude exposé-non exposé",
            "C: Etude transversale",
            "D: Un essai clinique",
            "E: Une série de cas"
        ],
        "resposta_correta": "A"
    },
    {
        "c": "Questão 13",
        "quizz_question": "Parmi les propositions suivantes, lesquelles font partie des critères d'inclusion de cette étude ?",
        "possible_answers": [
            "A: Etre une femme travaillant pour la fonction publique hospitalière en France",
            "B: Etre née entre 1950 et 1970",
            "C: Avoir complété les auto-questionnaires",
            "D: Avoir bénéficié des prises de sang entre 1990 et 1995",
            "E: Avoir donné des renseignements aux enquêteurs sur le statut ménopausique"
        ],
        "resposta_correta": "D"
    },
    {
        "c": "Questão 14",
        "quizz_question": "Dans cette étude, quel est l'évènement pathologique que présentent les sujets 'cas' et que ne présentent pas les sujets 'temoins'?",
        "possible_answers": [
            "A: le cancer du sein",
            "B: la ménopause",
            "C: le faible taux de vitamine D",
            "D: le surpoids",
            "E: le tabagisme"
        ],
        "resposta_correta": "A"
    },
    {
        "c": "Questão 15",
        "quizz_question": "Dans cette étude, quelles sont les mesures méthodologiques permettant la réduction de biais de sélection ?",
        "possible_answers": [
            "A: La réalisation des analyses des prélèvements sanguins dans un laboratoire n’ayant pas connaissance de la répartition des sujets entre les groupes cas et témoins.",
            "B: La réalisation des analyses des prélèvements sanguins dans un laboratoire n'ayant pas connaissance de la répartition des sujets entre les groupes cas et témoins.",
            "C: L'utilisation de techniques biologiques validées pour mesurer le taux de vitamine D.",
            "D: Le diagnostic de cancer du sein a été confirmé par un médecin",
            "E: Les sujets n'ayant pas rempli les questionnaires ou n'ayant pas eu de prélèvement sanguin ont été exclus de l'analyse"
        ],
        "resposta_correta": "A"
    }
]

In [25]:
final_results = []
partial_result = {
            "article": "",
            "section_counts": section_counts
        }
new_article = True

results_outputs =[]

In [26]:
for q in questions:   
    article = "2dff5573c7185251e19b4a301173e8f8bc85347c.pdf"
    quizz_question = ['quizz_question']
    possible_answers = ['possible_answers']


    question = f"""The following question is based on the provided article. 
        Please read the article carefully and determine which section of the article the question pertains to. Justify your choice by referring to specific parts of the article.

        Article: {article}

        Question: {quizz_question}
        Possible answers: {possible_answers}

        Identify the relevant section of the article and provide a brief justification."""

    template = """You will be provided with an article and a question related to it. 
    Your task is to read the article and determine the section that best answers the question. Provide a concise and specific justification for your choice.

    {context}

    Question: {question}

    Helpful Answer:"""
    QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

    qa = RetrievalQA.from_chain_type(
        llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=False,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}

    )

    result = qa({"query": question}, return_only_outputs=True)
    results_outputs.append(result['result'])

    # Carregar o pipeline de classificação de texto
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

    # Definir o texto e as categorias
    text = result['result']

    # Definir as seções como categorias
    labels = ["Introduction", "Methods", "Results", "Discussion", "Conclusion"]

    # Classificar o texto
    classification = classifier(text, labels)

    # Obter a seção mais provável
    predicted_section = classification['labels'][0]
    section_counts[predicted_section] += 1

final_results.append({
        "article": "2c5bee41e4ae9aa65206b113c7b84c8152cd4aed.pdf",
        "section_counts": section_counts
    })
print(final_results)

C:\Users\fiorentc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[{'article': '2c5bee41e4ae9aa65206b113c7b84c8152cd4aed.pdf', 'section_counts': {'Introduction': 0, 'Background': 0, 'Methods': 0, 'Results': 0, 'Discussion': 15, 'Conclusion': 0}}]


In [27]:
print(results_outputs)

["I'm happy to help!\n\nAfter carefully reading the provided article, I believe that the question pertains to Section 1 of the article. Here's why:\n\nThe article starts with a reference list (Section 1) that provides background information on various studies and research related to colorectal cancer screening and prevention. The references are cited throughout the article, including in the introduction, methods, results, and discussion sections.\n\nGiven that the question is based on the provided article, I think it's likely that the question is related to one of these referenced studies or concepts introduced in Section 1.", "I'm happy to help!\n\nAfter carefully reading the provided article, I believe that the question pertains to Section 1 of the article. Here's why:\n\nThe article starts with a reference list (Section 1) that provides background information on various studies and research related to colorectal cancer screening and prevention. The references are cited throughout th